In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create the table
cursor.execute('''
    CREATE TABLE purchases (
        store TEXT,
        date TEXT,
        item TEXT,
        price REAL,
        split INTEGER,
        sai_purchases REAL,
        tyler_purchases REAL,
        paid_by TEXT
    )
''')

In [3]:
# Load data from CSV file
csv_file = 'Transactions/' +'test.csv'
data = pd.read_csv(csv_file)


# Strip any leading or trailing whitespace from column names
data.columns = data.columns.str.strip()

# Ensure that column names match those in the DataFrame
data.rename(columns={
    'Store': 'store',
    'Date': 'date',
    'Item': 'item',
    'Price': 'price',
    'Split(1=Sai,2=Tyler,3=Split)': 'split',
    "Sai's Purchases": 'sai_purchases',
    "Tyler's Purchases": 'tyler_purchases',
    'Paid By': 'paid_by'
}, inplace=True)

# Insert data into the SQLite table
data.to_sql('purchases', conn, if_exists='append', index=False)

# Commit the transaction
conn.commit()

In [4]:
# Verify data has been inserted
cursor.execute('SELECT * FROM purchases Limit 5')
rows = cursor.fetchall()
for row in rows:
    print(row)


('Menards', '8/14/2024', 'Arnold Palmer', 2.99, 3, 1.495, 1.495, 'Sai')
('Menards', '8/14/2024', 'Air Circulator', 64.99, 2, 0.0, 64.99, 'Sai')
('Menards', '8/14/2024', 'Spray Paint x2', 9.98, 1, 9.98, 0.0, 'Sai')
('Menards', '8/14/2024', 'RW Impact Gloves', 9.99, 2, 0.0, 9.99, 'Sai')
('Menards', '8/14/2024', 'Recycling Bin', 4.7, 3, 2.35, 2.35, 'Sai')


In [5]:
# Define the function to calculate tax split
def calculate_tax_split(store, date):
    # Calculate total pre-tax amount
    cursor.execute('''
        SELECT SUM(price)
        FROM purchases
        WHERE store = ? AND date = ? AND item != 'Tax'
    ''', (store, date))
    total_pre_tax = cursor.fetchone()[0]
    
    if total_pre_tax is None:
        total_pre_tax = 0

    # Calculate total Sai's purchases pre-tax
    cursor.execute('''
        SELECT SUM(sai_purchases)
        FROM purchases
        WHERE store = ? AND date = ? AND item != 'Tax'
    ''', (store, date))
    total_sai = cursor.fetchone()[0]
    
    if total_sai is None:
        total_sai = 0

    # Calculate total Tyler's purchases pre-tax
    cursor.execute('''
        SELECT SUM(tyler_purchases)
        FROM purchases
        WHERE store = ? AND date = ? AND item != 'Tax'
    ''', (store, date))
    total_tyler = cursor.fetchone()[0]
    
    if total_tyler is None:
        total_tyler = 0

    # Get tax amount
    cursor.execute('''
        SELECT price
        FROM purchases
        WHERE store = ? AND date = ? AND item = 'Tax'
    ''', (store, date))
    tax = cursor.fetchone()[0]
    
    if tax is None:
        tax = 0

    # Calculate tax split
    if total_pre_tax > 0:
        sai_tax = total_sai / total_pre_tax * tax
        tyler_tax = total_tyler / total_pre_tax * tax
    else:
        sai_tax = 0
        tyler_tax = 0
    
    return (
        round(total_pre_tax, 2),
        round(total_sai, 2),
        round(total_tyler, 2),
        round(tax, 2),
        round(sai_tax,2),
        round(tyler_tax,2)
    )


def calculate_amounts_to_pay(store, date):
    # Run the tax split calculation
    results = calculate_tax_split(store, date)
    # print(results)
    total_pre_tax, total_sai, total_tyler, tax, sai_tax, tyler_tax = results

    # Calculate total amount Sai and Tyler owe
    total_sai_purchased = round(total_sai + sai_tax, 2)
    total_tyler_purchased = round(total_tyler + tyler_tax, 2)

    # Determine who paid
    cursor.execute('''
        SELECT paid_by
        FROM purchases
        WHERE store = ? AND date = ? AND item = 'Tax'
    ''', (store, date))
    paid_by = cursor.fetchone()[0]
    if paid_by=="Tyler":
        return paid_by, total_sai_purchased
    else: 
        return paid_by, total_tyler_purchased

# Example usage of the function
store_name = 'Menards'
transaction_date = '8/14/2024'
results = calculate_tax_split(store_name, transaction_date)
print(f"For the purchase @ {store_name} on {transaction_date}")
print(f"Total Pre-Tax: ${results[0]}")
print(f"Sai's Total Purchases: ${results[1]}")
print(f"Tyler's Total Purchases: ${results[2]}")
print(f"Total Tax: ${results[3]}")
print(f"Sai's Tax Split: ${results[4]}")
print(f"Tyler's Tax Split: ${results[5]}")

paid_by, due = calculate_amounts_to_pay(store_name, transaction_date)

# Print the amounts

if paid_by == 'Sai':
    print(f"Tyler owes Sai ${due}")
elif paid_by == 'Tyler':
    print(f"Sai owes tyler ${due}")



For the purchase @ Menards on 8/14/2024
Total Pre-Tax: $142.61
Sai's Total Purchases: $61.79
Tyler's Total Purchases: $80.82
Total Tax: $14.26
Sai's Tax Split: $6.18
Tyler's Tax Split: $8.08
Tyler owes Sai $88.9


In [6]:
def debt_balancing():
    # Get all unique (store, date) tuples
    cursor.execute('''
        SELECT DISTINCT store, date
        FROM purchases
    ''')
    unique_tuples = cursor.fetchall()
     #Initialize variables to track overall balance
    sai_owes_tyler_total = 0
    tyler_owes_sai_total = 0

    # Iterate over each unique (store, date) tuple
    for store, date in unique_tuples:
        paid_by, due = calculate_amounts_to_pay(store, date)
        
        if paid_by == 'Sai':
            tyler_owes_sai_total += due
            print(f"Tyler owes Sai ${due} for the transaction @ {store} on {date}")
        elif paid_by == 'Tyler':
            print(f"Sai owes Tyler ${due} for the transaction @ {store} on {date}")
            sai_owes_tyler_total += due
    print("\n")
    # Calculate the net balance
    if sai_owes_tyler_total > tyler_owes_sai_total:
        net_owes_tyler = round(sai_owes_tyler_total - tyler_owes_sai_total, 2)
        print(f"After balancing, Sai owes Tyler: ${net_owes_tyler}")
    elif tyler_owes_sai_total > sai_owes_tyler_total:
        net_owes_sai = round(tyler_owes_sai_total - sai_owes_tyler_total, 2)
        print(f"After balancing, Tyler owes Sai: ${net_owes_sai}")
    else:
        print("Neither owes anything; the balance is even.")


debt_balancing()

Tyler owes Sai $88.9 for the transaction @ Menards on 8/14/2024
Tyler owes Sai $10.89 for the transaction @ Menards on 8/12/2024
Tyler owes Sai $1.78 for the transaction @ IKEA on 8/13/2024
Sai owes Tyler $21.14 for the transaction @ Walmart on 8/14/2024
Sai owes Tyler $2.74 for the transaction @ IKEA on 8/15/2024
Tyler owes Sai $25.0 for the transaction @ ALDI on 8/12/2024
Tyler owes Sai $6.88 for the transaction @ Dollar Tree on 8/12/2024


After balancing, Tyler owes Sai: $109.57


In [7]:
conn.close()